In [1]:
print("all okay")

all okay


In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
import os
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
LANGSMITH_API_KEY = os.getenv("LANGSMITH_API_KEY")
LANGSMITH_TRACING_V2 = os.getenv("LANGSMITH_TRACING_V2")
LANGSMITH_PROJECT = os.getenv("LANGSMITH_PROJECT")
GROQ_API_KEY = os.getenv("GROQ_API_KEY")

Basic Graph

In [4]:
from langgraph.graph import StateGraph,START,END
from typing import TypedDict,Literal

In [5]:
class State(TypedDict):
    graph_state:str

In [6]:
def node_1(state:State) -> State:
    print("Node 1")
    response=state['graph_state'] + "I am"
    return {"graph_state":response}

def node_2(state:State) -> State:
    print("Node 2")
    response=state['graph_state'] + " Happy"
    return {"graph_state":response}

def node_3(state:State) -> State:
    print("Node 3")
    response=state['graph_state'] + " sad"
    return {"graph_state":response}

In [7]:
import random

def decision_node(state:State) -> Literal['happy_path','sad_path']:
    print("Decision Node")

    if random.random() > 0.5:
        return 'happy_path'
    return 'sad_path'

In [8]:
builder=StateGraph(State)
builder.add_node("node_1",node_1)
builder.add_node("node_2",node_2)
builder.add_node("node_3",node_3)
builder.add_edge(START,"node_1")
builder.add_node("decision_node",decision_node)

builder.add_conditional_edges(
    "node_1",
    decision_node,
    {
        'happy_path': "node_2",
        'sad_path': "node_3"
    }
)
builder.add_edge("node_2",END)
builder.add_edge("node_3",END)
app=builder.compile()

In [10]:
app.get_graph().print_ascii()

        +-----------+          
        | __start__ |          
        +-----------+          
               *               
               *               
               *               
          +--------+           
          | node_1 |           
          +--------+           
          .         .          
        ..           ..        
       .               .       
+--------+          +--------+ 
| node_2 |          | node_3 | 
+--------+          +--------+ 
          *         *          
           **     **           
             *   *             
          +---------+          
          | __end__ |          
          +---------+          


In [13]:
state={"graph_state":"Hi, this is Arun"}
app.invoke(state)

Node 1
Decision Node
Node 3


{'graph_state': 'Hi, this is ArunI am sad'}

In [15]:
app.invoke(state)

Node 1
Decision Node
Node 2


{'graph_state': 'Hi, this is ArunI am Happy'}

Now we will use agent with tools

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI

In [17]:
llm=ChatGoogleGenerativeAI(model='gemini-2.5-flash',temperature=0)
llm.invoke("what is the capital of india?")

AIMessage(content='The capital of India is **New Delhi**.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--4b57ddf4-d1ae-44da-9829-f367be9356a1-0', usage_metadata={'input_tokens': 8, 'output_tokens': 29, 'total_tokens': 37, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 20}})

In [18]:
def multiply(a:int,b:int):
    """This is the arithimetic function which multiplies two give integers a and b

    Args:
    a: first int
    b: second int
    """
    return a*b

tools=[multiply]
llm_with_tool=llm.bind_tools(tools)

In [19]:
from langgraph.graph import MessagesState
def tool_calling(state:MessagesState):
    response=llm_with_tool.invoke(state['messages'])
    return {"messages":[response]}

In [20]:
from langgraph.prebuilt import ToolNode
tool_node=ToolNode(tools)

In [54]:
def router_fun(state:MessagesState):
    last_message=state["messages"][-1]

    if last_message.tool_calls:
        return "tool"
    return END

In [55]:
builder1=StateGraph(MessagesState)
builder1.add_node("llm_with_tools",tool_calling)
builder1.add_node("tools",tool_node)
builder1.add_edge(START,"llm_with_tools")
builder1.add_conditional_edges(
    "llm_with_tools",
    router_fun,
    {
        "tool":"tools",
        END:END
    }

)
builder1.add_edge("tools",END)
app2=builder1.compile()


In [56]:
app2.get_graph().print_ascii()

        +-----------+     
        | __start__ |     
        +-----------+     
              *           
              *           
              *           
      +----------------+  
      | llm_with_tools |  
      +----------------+  
         ..        ..     
       ..            .    
      .               ..  
+-------+               . 
| tools |             ..  
+-------+            .    
         **        ..     
           **    ..       
             *  .         
         +---------+      
         | __end__ |      
         +---------+      


In [62]:
state={"messages":["what is 4 multiplied with 5"]}
state1={"messages":["how are you"]}

In [59]:
response=app2.invoke(state)

In [61]:
for msg in response["messages"]:
    msg.pretty_print()

================================ Human Message =================================

what is 4 multiplied with 5
================================== Ai Message ==================================
Tool Calls:
  multiply (32faf3b7-21b9-46cc-b440-3af85fdaae12)
 Call ID: 32faf3b7-21b9-46cc-b440-3af85fdaae12
  Args:
    b: 5
    a: 4
================================= Tool Message =================================
Name: multiply

20


Now we will create REACT structure

In [63]:
response=app2.invoke(state1)
for msg in response["messages"]:
    msg.pretty_print()

================================ Human Message =================================

how are you
================================== Ai Message ==================================

I'm doing well, thank you! How can I help you today?


In [66]:
def add(a:int,b:int):
    """This function adds two intergers a and b.

    Args:
    a : first integer
    b: second integer
    """
    return a+b

def divide(a:int,b:int):
    """This function divides two intergers a and b.

    Args:
    a : first integer
    b: second integer
    """
    return a/b

In [81]:
tool_upgrade=[multiply,add,divide]
tool_node_upgrade=ToolNode(tool_upgrade)
llm_with_tool_upgrade=llm.bind_tools(tool_upgrade)
def tool_calling_upgrade(state:MessagesState):
    response=llm_with_tool_upgrade.invoke(state['messages'])
    return {"messages":[response]}

In [82]:
builder2=StateGraph(MessagesState)
builder2.add_node("llm_with_tool_upgrade",tool_calling_upgrade)
builder2.add_node("tools_upgrate",tool_node_upgrade)
builder2.add_edge(START,"llm_with_tool_upgrade")
builder2.add_conditional_edges(
    "llm_with_tool_upgrade",
    router_fun,
    {
        "tool":"tools_upgrate",
        END:END
    }
)
builder2.add_edge("tools_upgrate","llm_with_tool_upgrade")

app3=builder2.compile()

In [83]:
app3.get_graph().print_ascii()

            +-----------+               
            | __start__ |               
            +-----------+               
                  *                     
                  *                     
                  *                     
      +-----------------------+         
      | llm_with_tool_upgrade |         
      +-----------------------+         
           ...         ...              
          .               .             
        ..                 ..           
+---------+           +---------------+ 
| __end__ |           | tools_upgrate | 
+---------+           +---------------+ 


In [84]:
state3={"messages":["Add 3 and 4. Multiply the output by 2. Divide the output by 5"]}

In [87]:
response=app3.invoke(state3)

In [88]:
for msg in response["messages"]:
    msg.pretty_print()

================================ Human Message =================================

Add 3 and 4. Multiply the output by 2. Divide the output by 5
================================== Ai Message ==================================
Tool Calls:
  add (fdb41117-d057-462f-8400-65a413d216b3)
 Call ID: fdb41117-d057-462f-8400-65a413d216b3
  Args:
    b: 4
    a: 3
================================= Tool Message =================================
Name: add

7
================================== Ai Message ==================================
Tool Calls:
  multiply (5eadf186-b77a-498c-916b-3cd704496fe8)
 Call ID: 5eadf186-b77a-498c-916b-3cd704496fe8
  Args:
    b: 2
    a: 7
================================= Tool Message =================================
Name: multiply

14
================================== Ai Message ==================================
Tool Calls:
  divide (0dee0446-a1a3-4ef6-ab0f-5be182e2d1e9)
 Call ID: 0dee0446-a1a3-4ef6-ab0f-5be182e2d1e9
  Args:
    b: 5
    a: 14
====================

In [89]:
response=app3.invoke(state1)

In [90]:
for msg in response["messages"]:
    msg.pretty_print()

================================ Human Message =================================

how are you
================================== Ai Message ==================================

I am an AI assistant, I do not have personal feelings. How can I help you today?
